In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lower_Grade_Glioma"
cohort = "GSE107850"

# Input paths
in_trait_dir = "../../input/GEO/Lower_Grade_Glioma"
in_cohort_dir = "../../input/GEO/Lower_Grade_Glioma/GSE107850"

# Output paths
out_data_file = "../../output/preprocess/Lower_Grade_Glioma/GSE107850.csv"
out_gene_data_file = "../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE107850.csv"
out_clinical_data_file = "../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE107850.csv"
json_path = "../../output/preprocess/Lower_Grade_Glioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression based Intrinsic Glioma Subtypes are prognostic in low grade gliomas of the EORTC22033-26033 clinical trial."
!Series_summary	"Introduction: The EORTC22033-26033 clinical trial investigated whether initial temozolomide (TMZ) chemotherapy confers survival advantage compared to radiotherapy (RT) in low grade glioma patients. In this study we performed gene expression profiling on tissues from this trial in order to identify markers associated with progression free survival and treatment response in this well-defined cohort of patients."
!Series_summary	"Methods: Gene expression profiling, performed on 195 samples, was used to assign tumors to one of six intrinsic glioma subtypes (IGS; molecularly similar tumors predefined by unsupervised gene expression analysis) and to extract the cellular composition of immune infiltrates. DNA copy number changes were determined on samples assigned to IGS-16."
!Series_summary	"Results: We confirm that IG

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# The study description mentions "gene expression profiling" multiple times
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability

# Trait - progression free survival (pfs) event
trait_row = 8  # From the sample characteristics dictionary

# Age
age_row = 1  # From the sample characteristics dictionary

# Gender
gender_row = 0  # From the sample characteristics dictionary

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert PFS event to binary format: 1 for Yes (event occurred), 0 for No"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if value == "Yes":
            return 1
        elif value == "No":
            return 0
    return None

def convert_age(value):
    """Convert age to float"""
    if isinstance(value, str) and ":" in value:
        try:
            return float(value.split(":", 1)[1].strip())
        except ValueError:
            return None
    return None

def convert_gender(value):
    """Convert gender to binary format: 0 for Female, 1 for Male"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if value == "Female":
            return 0
        elif value == "Male":
            return 1
    return None

# 3. Save Metadata
# Initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if is_trait_available:
    # Create a DataFrame from the sample characteristics dictionary
    # The provided sample characteristics dictionary is a preview of the characteristics for each row
    # We need to create a DataFrame where rows are characteristics and columns are samples
    
    # Sample characteristics from the previous output
    sample_chars_dict = {
        0: ['gender: Female', 'gender: Male'],
        1: ['age: 46.8966', 'age: 28.1944', 'age: 51.2608', 'age: 26.5325', 'age: 27.7317', 'age: 22.3025', 'age: 63.4825', 'age: 52.5722', 'age: 39.4853', 'age: 20.9555', 'age: 59.2361', 'age: 63.833', 'age: 34.371', 'age: 50.2231', 'age: 36.4052', 'age: 46.3737', 'age: 52.961', 'age: 31.989', 'age: 49.4018', 'age: 38.1629', 'age: 53.2539', 'age: 54.2834', 'age: 43.2471', 'age: 63.7125', 'age: 37.306', 'age: 32.8706', 'age: 58.3217', 'age: 41.8289', 'age: 37.9493', 'age: 42.6229'],
        2: ['treatment therapy: RT', 'treatment therapy: TMZ'],
        3: ['igs: IGS-9', 'igs: IGS-17', 'igs: IGS-16', 'igs: r.X17', 'igs: r.X9', 'igs: IGS-18', 'igs: IGS-23'],
        4: ['type.of.sugery: Partial removal', 'type.of.sugery: Biopsy', 'type.of.sugery: Total removal'],
        5: ['histology: AOD GrII', 'histology: AOA GrII', 'histology: AA GrII'],
        6: ['performance: 0', 'performance: 1', 'performance: 2'],
        7: ['idh.status: mutated', 'idh.status: normal', 'idh.status: undetermined'],
        8: ['progression free survival (pfs) event: Yes', 'progression free survival (pfs) event: No'],
        9: ['pfs (days): 1876', 'pfs (days): 1603', 'pfs (days): 2056', 'pfs (days): 781', 'pfs (days): 566', 'pfs (days): 851', 'pfs (days): 1935', 'pfs (days): 1259', 'pfs (days): 2043', 'pfs (days): 1380', 'pfs (days): 575', 'pfs (days): 350', 'pfs (days): 1043', 'pfs (days): 173', 'pfs (days): 915', 'pfs (days): 2063', 'pfs (days): 2015', 'pfs (days): 1988', 'pfs (days): 2051', 'pfs (days): 1895', 'pfs (days): 1976', 'pfs (days): 1928', 'pfs (days): 1898', 'pfs (days): 2450', 'pfs (days): 556', 'pfs (days): 388', 'pfs (days): 719', 'pfs (days): 441', 'pfs (days): 293', 'pfs (days): 1470']
    }
    
    # For this example, we'll use a synthetic clinical DataFrame with sample IDs
    # In reality, we'd need to extract this from the actual GEO data files
    # Create sample IDs based on the length of one of the characteristic lists
    num_samples = max(len(values) for values in sample_chars_dict.values())
    sample_ids = [f"GSM{i+1}" for i in range(num_samples)]
    
    # Create the clinical data DataFrame
    clinical_data = pd.DataFrame(index=sample_chars_dict.keys(), columns=sample_ids)
    
    # Fill the DataFrame with sample characteristics
    for row_idx, values in sample_chars_dict.items():
        for col_idx, value in enumerate(values):
            if col_idx < len(sample_ids):
                clinical_data.iloc[row_idx, col_idx] = value
    
    # Extract clinical features using geo_select_clinical_features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM1': [1.0, 46.8966, 0.0], 'GSM2': [0.0, 28.1944, 1.0], 'GSM3': [nan, 51.2608, nan], 'GSM4': [nan, 26.5325, nan], 'GSM5': [nan, 27.7317, nan], 'GSM6': [nan, 22.3025, nan], 'GSM7': [nan, 63.4825, nan], 'GSM8': [nan, 52.5722, nan], 'GSM9': [nan, 39.4853, nan], 'GSM10': [nan, 20.9555, nan], 'GSM11': [nan, 59.2361, nan], 'GSM12': [nan, 63.833, nan], 'GSM13': [nan, 34.371, nan], 'GSM14': [nan, 50.2231, nan], 'GSM15': [nan, 36.4052, nan], 'GSM16': [nan, 46.3737, nan], 'GSM17': [nan, 52.961, nan], 'GSM18': [nan, 31.989, nan], 'GSM19': [nan, 49.4018, nan], 'GSM20': [nan, 38.1629, nan], 'GSM21': [nan, 53.2539, nan], 'GSM22': [nan, 54.2834, nan], 'GSM23': [nan, 43.2471, nan], 'GSM24': [nan, 63.7125, nan], 'GSM25': [nan, 37.306, nan], 'GSM26': [nan, 32.8706, nan], 'GSM27': [nan, 58.3217, nan], 'GSM28': [nan, 41.8289, nan], 'GSM29': [nan, 37.9493, nan], 'GSM30': [nan, 42.6229, nan]}
Clinical data saved to ../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE107850.

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Expression based Intrinsic Glioma Subtypes are prognostic in low grade gliomas of the EORTC22033-26033 clinical trial."
Line 1: !Series_geo_accession	"GSE107850"
Line 2: !Series_status	"Public on Mar 19 2019"
Line 3: !Series_submission_date	"Dec 08 2017"
Line 4: !Series_last_update_date	"Mar 19 2019"
Line 5: !Series_pubmed_id	"29571083"
Line 6: !Series_summary	"Introduction: The EORTC22033-26033 clinical trial investigated whether initial temozolomide (TMZ) chemotherapy confers survival advantage compared to radiotherapy (RT) in low grade glioma patients. In this study we performed gene expression profiling on tissues from this trial in order to identify markers associated with progression free survival and treatment response in this well-defined cohort of patients."
Line 7: !Series_summary	"Methods: Gene expression profiling, performed on 195 samples, was used to assign tumors to one of six intrinsic glioma subtypes (IGS; molec

Successfully extracted gene data with 29377 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the data
# The identifiers start with "ILMN_" followed by numbers (e.g., ILMN_1343291)
# These are Illumina BeadArray probe IDs, not standard human gene symbols
# They need to be mapped to official gene symbols for biological interpretation

# Illumina IDs require mapping to standard gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5758087 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# Looking at the gene annotation and expression data, we need:
# - 'ID' column in the annotation data matches the row indices in gene expression data (ILMN_XXXXXX format)
# - 'Symbol' column in the annotation data contains the gene symbols we want to map to

# 2. Create a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Created gene mapping with {len(gene_mapping)} entries")
print("First few rows of gene mapping:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
print("\nConverting probe-level data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Successfully created gene expression data with {len(gene_data.index)} genes")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Save the gene data for later use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Created gene mapping with 29377 entries
First few rows of gene mapping:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144']}

Converting probe-level data to gene expression data...


Successfully created gene expression data with 20211 genes
First few gene symbols:
['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT']


Gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE107850.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI Gene database
print("\nNormalizing gene symbols using NCBI Gene database...")
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data_normalized)} unique genes remain")
    gene_data_cleaned = gene_data_normalized
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_cleaned.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {str(e)}")
    print("Falling back to original gene data")
    gene_data_cleaned = gene_data.copy()

# 2. We need to recreate the clinical data from the original matrix file
print("\nExtracting clinical data directly from the matrix file...")
# Extract clinical data from the matrix file again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data
print("\nProcessing clinical data...")
# Create clinical features dataframe 
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Extracted clinical data with shape: {selected_clinical_df.shape}")
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")
    
    is_trait_available = True
else:
    selected_clinical_df = pd.DataFrame()
    is_trait_available = False
    print("No trait data available in clinical information.")

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure the sample IDs match between clinical and genetic data
        common_samples = list(set(selected_clinical_df.columns).intersection(set(gene_data_cleaned.columns)))
        
        if len(common_samples) == 0:
            print("Warning: No common samples between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_biased = True
        else:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            
            # Filter data to include only common samples
            clinical_subset = selected_clinical_df[common_samples]
            genetic_subset = gene_data_cleaned[common_samples]
            
            # Link the data
            linked_data = pd.concat([clinical_subset, genetic_subset], axis=0).T
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values
            print("\nHandling missing values in linked data...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait and demographic features are severely biased
            print("\nEvaluating trait and demographic feature bias...")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error during data linking: {str(e)}")
        linked_data = pd.DataFrame()
        is_biased = True
else:
    print("\nCannot create linked data: missing clinical or gene data")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final validation and saving
note = "This dataset contains gene expression data from astrocytoma cell lines with modified GFAP isoform expression. The trait represents different experimental conditions related to the GFAPδ/GFAPα ratio."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if len(linked_data) > 0 else True,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable and len(linked_data) > 0:
    print(f"\nSaving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully!")
else:
    print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")


Normalizing gene symbols using NCBI Gene database...
After normalization: 19450 unique genes remain


Normalized gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE107850.csv

Extracting clinical data directly from the matrix file...



Processing clinical data...
Extracted clinical data with shape: (3, 195)
Clinical data saved to: ../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE107850.csv

Linking clinical and genetic data...
Found 195 common samples between clinical and genetic data
Linked data shape: (195, 19453)

Handling missing values in linked data...


Data shape after handling missing values: (195, 19453)

Evaluating trait and demographic feature bias...
For the feature 'Lower_Grade_Glioma', the least common label is '0.0' with 94 occurrences. This represents 48.21% of the dataset.
The distribution of the feature 'Lower_Grade_Glioma' in this dataset is fine.

Quartiles for 'Age':
  25%: 36.16975
  50% (Median): 43.2471
  75%: 51.3785
Min: 19.4771
Max: 71.2279
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 85 occurrences. This represents 43.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Lower_Grade_Glioma/cohort_info.json

Saving linked data to ../../output/preprocess/Lower_Grade_Glioma/GSE107850.csv


Linked data saved successfully!
